In [2]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator
from sklearn.datasets import make_regression
from sklearn.datasets import make_friedman1
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

Вставьте код в класс, приведённый ниже. В итоге должен получиться бустинг над деревьями.

In [118]:
class SimpleGB(BaseEstimator):
    def __init__(self, tree_params_dict, iters=100, tau=1e-1):
        self.tree_params_dict = tree_params_dict
        self.iters = iters
        self.tau = tau
        
    def fit(self, X_data, y_data):
        self.estimators = []
        curr_pred = 0
        y = y_data
        for iter_num in np.arange(self.iters):
            # insert code here
            y -= curr_pred
            algo = DecisionTreeRegressor(**self.tree_params_dict).fit(X_data, y)
            self.estimators.append(algo)
            step = 1 if iter_num == 0 else (1 / iter_num) ** (self.tau / iter_num)
            curr_pred = step * algo.predict(X_data)
        return self 
    
    def predict(self, X_data):
        res = np.zeros(X_data.shape[0])
        for estimator in self.estimators:
            res += estimator.predict(X_data)
        return res

## Проверка качества полученного класса

Можете поиграться с параметрами, посмотрим, у кого самое лучшее качество получится

Сгенерируем данные с линейной зависимостью

In [15]:
X_data, y_data = make_regression(n_samples=1000, noise=10, n_features=10, random_state=42)

In [16]:
algo = SimpleGB(
    tree_params_dict={
        'max_depth':4
    },
    iters=100,
    tau = 0.1
)

In [17]:
np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-4856.018795807654

In [18]:
np.mean(cross_val_score(DecisionTreeRegressor(max_depth=4), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-7550.7275207710281

In [19]:
np.mean(cross_val_score(LinearRegression(), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-95.517114056519929

на реально-линейной зависимости не дотягивает до линейной модели, но уже существенно лучше чем одно дерево.

И аналогично, сгенерируем нелинейную зависимость

In [81]:
X_data, y_data = make_friedman1(n_samples=1000, noise=10, n_features=10, random_state=42)

In [82]:
algo = SimpleGB(
    tree_params_dict={
        'max_depth': 1
    },
    iters=100,
    tau=0.1
)

In [83]:
np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-118.86403546659747

In [73]:
np.mean(cross_val_score(DecisionTreeRegressor(max_depth=6), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-137.94823601075473

In [72]:
np.mean(cross_val_score(LinearRegression(), X_data, y_data, cv=5, scoring='neg_mean_squared_error'))

-108.80318041227227

### Финальный код оценки качества (вам нужно GB реализовать и подобрать хорошие tree_params)

In [143]:
tree_params_dict = {'max_depth' : 1, 'random_state' : 1, 'min_samples_leaf' : 5} # TODO
iters = 11 #TODO
tau = 0.03 # TODO

algo = SimpleGB(
    tree_params_dict=tree_params_dict,
    iters=iters,
    tau=tau
)
print(np.mean(cross_val_score(algo, X_data, y_data, cv=5, scoring='neg_mean_squared_error')))

-110.10842445
